# Imports

In [ ]:
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from importlib import reload
import software.analysis as a
reload(a)

In [ ]:
local_analysis = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/")

In [ ]:
csv_path = local_analysis / "20230411_full_posit_results/analysis"

# Load Processed Results

In [ ]:
df = pd.read_csv(csv_path / "20240117_03_with_info.csv", index_col=0)

In [ ]:
missing_cmpd_ids = set(df.Compound_ID.unique()) - set(df.Reference_Ligand.unique())

In [ ]:
missing_cmpd_ids

# Load Mpro Soaks and get correct data info

In [ ]:
soaks = pd.read_csv("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/fragalysis_downloads/20240129_fragalysis_download/extra_files/Mpro_soaks.csv")

In [ ]:
filtered = soaks[(-soaks["Compound ID"].isna())&(-soaks["Sample Name"].isna())&(soaks["Data Collection Outcome"] == "success")]

In [ ]:
filtered_sorted = filtered.sort_values("Sample Name", ascending=True)

In [ ]:
p_only = filtered_sorted[filtered_sorted["Sample Name"].str.contains("Mpro-P")]

In [ ]:
p_only = p_only[p_only["Compound ID"] != "Apo"]
p_only = p_only[p_only["Refinement Outcome"].isin(["5 - Deposition ready", "6 - Deposited"])]

In [ ]:
p_only.groupby("Compound ID").count()

In [ ]:
p_only[p_only["Compound ID"] == "ERI-UCB-ce40166b-17"].columns

In [ ]:
p_only[p_only["Compound ID"] == "ERI-UCB-ce40166b-17"][["Sample Name", "Data Collection Outcome", "Refinement Outcome"]]

In [ ]:
cmpd_to_frag = {cmpd: frag for cmpd, frag in zip(p_only["Compound ID"], p_only["Sample Name"])}

In [ ]:
cmpd_to_frag

In [ ]:
for id in missing_cmpd_ids:
    print(cmpd_to_frag.get(id, None))

In [ ]:
len(df.Reference_Ligand.unique())

In [ ]:
set(df.Reference_Ligand) - set(df.Compound_ID)

In [ ]:
missing = [lig for lig in df.Reference_Ligand.unique() if not lig in cmpd_to_frag.keys()]

In [ ]:
missing

In [ ]:
missing = [lig for lig in df.Compound_ID.unique() if not lig in cmpd_to_frag.keys()]

In [ ]:
missing

In [ ]:
soaks[soaks["Compound ID"] == 'LON-WEI-adc59df6-47'][["Data Collection Outcome", "Status", "Sample Name", "Refinement Outcome", "SMILES"]]

In [ ]:
soaks[soaks["Compound ID"] == 'LON-WEI-adc59df6-47']["SMILES"].to_list()[0]

In [ ]:
import rdkit.Chem as chem

In [ ]:
chem.MolFromSmiles('CCC(=O)N(C(C(=O)NCCC1=CC(F)=CC=C1)C1=CN=CC=C1)C1=CC=C(C=C1)C(C)(C)C')

In [ ]:
soaks[soaks["Compound ID"] == 'EDJ-MED-705e09b8-1'][["Data Collection Outcome", "Status", "Sample Name", "Refinement Outcome", "SMILES"]]

In [ ]:
soaks[soaks["Refinement Outcome"] == "7 - Analysed & Rejected"]

## So since there are discrepancies in the system of record I will manually add the two missing ones

In [ ]:
cmpd_to_frag['LON-WEI-adc59df6-47'] = 'Mpro-P0047'
cmpd_to_frag['EDJ-MED-705e09b8-1'] = 'Mpro-P2607'

In [ ]:
len(cmpd_to_frag)

In [ ]:
len(set(cmpd_to_frag.values()))

In [ ]:
len(set(cmpd_to_frag.keys()))

## So now what's missing from `Reference Ligs` that is in cmpd to frag?

In [ ]:
missing = [lig for lig in cmpd_to_frag.keys() if not lig in df.Reference_Ligand.unique()]

In [ ]:
missing

In [ ]:
cmpd_to_frag[missing[0]]

in mpro_compound_tracker.csv this is 'EDJ-MED-976a33d5-1'

In [ ]:
'Mpro-P2724' in df.Structure_Name

In [ ]:
'EDJ-MED-976a33d5-1' in df.Reference_Ligand

In [ ]:
'EDJ-MED-976a33d5-1' in df.Compound_ID

maybe it's new?

In [ ]:
soaks[soaks['Compound ID'] == 'EDJ-MED-976a33d5-1']

In [ ]:
import numpy as np
filteredv2 = soaks[(-soaks["Fragalysis Link"].isna())&(soaks["Sample Name"].str.contains("Mpro-P"))]

In [ ]:
filteredv2.groupby(["Compound ID"]).count()

In [ ]:
newdata = filteredv2["Sample Name"].unique()

In [ ]:
whatarethese = [lig for lig in newdata if not lig in cmpd_to_frag.values()]

In [ ]:
whatarethese

In [ ]:
soaks[soaks["Sample Name"] == "Mpro-P0050"]["Fragalysis Link"].to_list()

In [ ]:
soaks[soaks["Sample Name"] == "Mpro-P0050"].columns

In [ ]:
soaks[soaks["Sample Name"] == "Mpro-P0050"][["Refinement Outcome", "Zenodo Status", "Postera series"]]

ok this is a terrifying rabbit hole

# Save new cmpd_to_frag data

In [ ]:
import yaml

In [ ]:
output = Path("../../data/")

In [ ]:
output.mkdir()

In [ ]:
with open(output / '202240129_p_only_cmpd_to_frag.yaml', 'w') as f:
    yaml.safe_dump(cmpd_to_frag, f)

# check if all those datasets actually exist in the output

In [ ]:
aligned_path = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/fragalysis_downloads/20240129_fragalysis_download/aligned/")

In [ ]:
p_structures = [pdb for pdb in aligned_path.glob("Mpro-P*/*.pdb")]

In [ ]:
len(p_structures)

In [ ]:
datasets = {}
for path in p_structures:
    name = path.stem.split("_")[0]
    new_path = datasets.get(name, None)
    if not new_path:
        datasets[name] = path

In [ ]:
len(datasets)

In [ ]:
missing = [dataset for dataset in cmpd_to_frag.values() if not dataset in datasets.keys()]

In [ ]:
missing

## make sure all the pdb files exist

In [ ]:
all([path.exists for path in datasets.values()])

## they do!

## make sure all sdfs exist

In [ ]:
all([path.with_suffix(".sdf") for path in datasets.values()])

## they do! woo!